In [1]:
import pandas as pd
import numpy as np

In [2]:
df_IRI = pd.read_excel('Data/2022 VNL/IRI VS POL.xlsx', sheet_name='IRI Set1')
df_IRI.head()

,No.,Space,Action,Erros,Attempts,Score,Unnamed: 6
0,2.0,Y,JS,NaN,NaN,NaN,NaN
1,22.0,1,B,NaN,NaN,1.0,NaN
2,2.0,Y,JS,NaN,NaN,NaN,NaN
3,2.0,8,D,1.0,NaN,NaN,NaN
4,8.0,8,R,NaN,NaN,NaN,NaN


In [3]:
df_IRI = df_IRI.drop(columns=['Unnamed: 6'])

---
* NaN 用 0 取代
* 去除最後一筆資料 (結算最後比分)
* 加上隊伍資訊
* Action == OP 的 column 去除，reset index

In [4]:
df_IRI = df_IRI.fillna(0)
df_IRI = df_IRI.drop(len(df_IRI)-1)
df_IRI.insert(0, column='Team', value=[1 for i in range(len(df_IRI))])
df_IRI = df_IRI.drop(df_IRI[df_IRI['Action'] == 'OP'].index)
df_IRI = df_IRI.reset_index(drop=True)

In [5]:
df_IRI = df_IRI.astype({'No.': 'int32'})
df_IRI = df_IRI.astype({'No.': 'str'})

In [6]:
df_IRI['Space'].replace({'M': 9, 'Y': 10, 'L': 11, 'FZ': 12}, inplace=True)
df_IRI['Action'].replace({'A': 1, 'B': 2, 'D': 3, 'FS': 4, 'G': 5, 'JS': 6, 'OP': 7, 'R': 8}, inplace=True)

In [7]:
df_IRI.head()

,Team,No.,Space,Action,Erros,Attempts,Score
0,1,2,10,6,0.0,0.0,0.0
1,1,22,1,2,0.0,0.0,1.0
2,1,2,10,6,0.0,0.0,0.0
3,1,2,8,3,1.0,0.0,0.0
4,1,8,8,8,0.0,0.0,0.0


---
* Team、No. 做 one-hot encoding
* Space、Action 比照羽球 paper 辦理

In [8]:
space_col = ['Space']
action_col = ['Action']
other_col = [c for c in df_IRI.columns if(c not in space_col and c not in action_col)]

print(space_col)
print(action_col)
print(other_col)

['Space']
['Action']
['Team', 'No.', 'Erros', 'Attempts', 'Score']


In [9]:
# 分開並在訓練階段 space 和 action 單獨丟進自己的 embedding layer
space_encoding = df_IRI[space_col].values.astype('float32')
action_encoding = df_IRI[action_col].values.astype('float32')
other_encoding = df_IRI[other_col].values.astype('float32')

---
按照一球把data分開 (好像也無法)
